In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("SparkSession") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/16 16:16:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

In [5]:
ratingDF = spark.read.option("header","true").csv("spark-data/ratings.csv")
windowPartition7 = Window.partitionBy("movieId","rating").orderBy(col("userId").cast("long")).rangeBetween(Window.currentRow, 7)
windowPartition15 = Window.partitionBy("movieId","rating").orderBy(col("userId").cast("long")).rangeBetween(Window.currentRow, 15)
windowPartition30 = Window.partitionBy("movieId","rating").orderBy(col("userId").cast("long")).rangeBetween(Window.currentRow, 30)


(ratingDF
.withColumn("rating_7",count("rating").over(windowPartition7))
.withColumn("rating_15",count("rating").over(windowPartition15))
.withColumn("rating_30",count("rating").over(windowPartition30))
.explain("Formatted"))

== Physical Plan ==
AdaptiveSparkPlan (13)
+- Project (12)
   +- Window (11)
      +- Sort (10)
         +- Project (9)
            +- Window (8)
               +- Sort (7)
                  +- Project (6)
                     +- Window (5)
                        +- Sort (4)
                           +- Exchange (3)
                              +- Project (2)
                                 +- Scan csv  (1)


(1) Scan csv 
Output [4]: [userId#17, movieId#18, rating#19, timestamp#20]
Batched: false
Location: InMemoryFileIndex [file:/Users/shivam/Downloads/Spark/Spark_Optimization_new/Day3/spark-data/ratings.csv]
ReadSchema: struct<userId:string,movieId:string,rating:string,timestamp:string>

(2) Project
Output [5]: [userId#17, movieId#18, rating#19, timestamp#20, cast(userId#17 as bigint) AS _w0#24L]
Input [4]: [userId#17, movieId#18, rating#19, timestamp#20]

(3) Exchange
Input [5]: [userId#17, movieId#18, rating#19, timestamp#20, _w0#24L]
Arguments: hashpartitioning(movieId#18, ra